In [3]:
import os

In [4]:
def download_images(images_directory):
    all_images_paths = []
    images_directories = os.listdir(images_directory)
    for directory in images_directories:
        directory_relaitve_path = f"{images_directory}/{directory}"
        files = os.listdir(directory_relaitve_path)
        for file in files:
            if file.endswith(".png"):
                all_images_paths.append(f"{images_directory}/{directory}/{file}")
            else:
                 pass
    return all_images_paths

In [5]:
img_paths = download_images("/Users/ryanhughes/Desktop/Aicore/Airbnb/Airbnb_Git_Repo/AirbnbDataSci/images")

In [13]:
import cv2

In [17]:
cv2.imread(img_paths[0]).shape

(618, 720, 3)

In [18]:
def create_folder(dest_path):
    try:
        os.mkdir(dest_path)
    except FileExistsError:
        pass

In [19]:
create_folder("/Users/ryanhughes/Desktop/Aicore/Airbnb/Airbnb_Git_Repo/AirbnbDataSci/processed_images")

In [10]:
import glob
import cv2

In [9]:
for i in glob.glob("/Users/ryanhughes/Desktop/Aicore/Airbnb/Airbnb_Git_Repo/AirbnbDataSci/images/*/*"):
    print(i)


/Users/ryanhughes/Desktop/Aicore/Airbnb/Airbnb_Git_Repo/AirbnbDataSci/images/248368e6-c98a-40dd-8c06-9a2456cbfeb9/248368e6-c98a-40dd-8c06-9a2456cbfeb9-c.png
/Users/ryanhughes/Desktop/Aicore/Airbnb/Airbnb_Git_Repo/AirbnbDataSci/images/248368e6-c98a-40dd-8c06-9a2456cbfeb9/248368e6-c98a-40dd-8c06-9a2456cbfeb9-b.png
/Users/ryanhughes/Desktop/Aicore/Airbnb/Airbnb_Git_Repo/AirbnbDataSci/images/248368e6-c98a-40dd-8c06-9a2456cbfeb9/248368e6-c98a-40dd-8c06-9a2456cbfeb9-a.png
/Users/ryanhughes/Desktop/Aicore/Airbnb/Airbnb_Git_Repo/AirbnbDataSci/images/248368e6-c98a-40dd-8c06-9a2456cbfeb9/248368e6-c98a-40dd-8c06-9a2456cbfeb9-e.png
/Users/ryanhughes/Desktop/Aicore/Airbnb/Airbnb_Git_Repo/AirbnbDataSci/images/248368e6-c98a-40dd-8c06-9a2456cbfeb9/248368e6-c98a-40dd-8c06-9a2456cbfeb9-d.png
/Users/ryanhughes/Desktop/Aicore/Airbnb/Airbnb_Git_Repo/AirbnbDataSci/images/244d52d5-9016-4309-a2a6-cb02b560f51b/244d52d5-9016-4309-a2a6-cb02b560f51b-d.png
/Users/ryanhughes/Desktop/Aicore/Airbnb/Airbnb_Git_Repo/Ai

In [11]:
word = "penis breath"
word[:-4]

'penis br'